<a href="https://colab.research.google.com/github/M0STAFA-MO/Facial-Emotion-Recognition/blob/main/Emotion%20Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Emotion Dataset Paths with Image Counts
# @markdown 📁 **Shows number of images per emotion class**
# @markdown
import os

emotion_paths = {
    "Angry": "/content/drive/MyDrive/Colab Notebooks/project/Emotions-Data/Angry",
    "Happy": "/content/drive/MyDrive/Colab Notebooks/project/Emotions-Data/Happy",
    'surprise': '/content/drive/MyDrive/Colab Notebooks/project/Emotions-Data/Surprise',
    'fear': "/content/drive/MyDrive/Colab Notebooks/project/Emotions-Data/Fear",
    'neutral': '/content/drive/MyDrive/Colab Notebooks/project/Emotions-Data/Neutral',

}

print("📊 Emotion Dataset Statistics:")
print("-" * 40)
for emotion, path in emotion_paths.items():
        num_images = len([f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"{emotion}: {num_images} images")
print("-" * 40)

📊 Emotion Dataset Statistics:
----------------------------------------
Angry: 310 images
Happy: 309 images
surprise: 310 images
fear: 310 images
neutral: 238 images
----------------------------------------


In [ ]:
# @title # Process multiple images in batches
# @markdown 🎯 Why Transfer Learning?
# @markdown Since my dataset is small, transfer learning is the optimal strategy. Here's why:
# @markdown
# @markdown - **Pre-trained Knowledge**: ResNet50 was trained on 1.2 million images (ImageNet), learning hierarchical features like edges, textures, and object parts. You reuse these universal features instead of training from scratch.
# @markdown ```python
# @markdown from keras.applications.resnet import ResNet50
# @markdown ```
# @markdown
# @markdown - **Avoid Overfitting**: With limited data, training a deep model from scratch would likely memorize the data (overfit). Transfer learning reduces this risk by leveraging pre-learned features.
# @markdown
# @markdown - **Faster Convergence**: You start with meaningful weights, requiring fewer epochs and computational resources.
# @markdown
# @markdown 🔍 **Why These Parameters?**
# @markdown After testing multiple models and configurations:
# @markdown
# @markdown - **ResNet50** outperformed others (VGG16, MobileNet) in accuracy while maintaining reasonable training time.
# @markdown
# @markdown - **Input shape (220, 220, 3)** was chosen because:
# @markdown ```python
# @markdown base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(220, 220, 3))
# @markdown ```
# @markdown - **Input shape (220, 220, 3)** was chosen because:
# @markdown   - 📉 Lower resolutions (< 220px) lost important facial details
# @markdown   - 📈 Higher resolutions (> 220px) increased compute cost without improving accuracy
# @markdown   - ✅ 220×220 preserved features optimally for emotion recognition tasks
from keras.applications.resnet import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(220, 220, 3))

In [ ]:
# @title # Process multiple images in batches
# @markdown 🔄 **Batch Processing Pipeline**
# @markdown
# @markdown - **Efficient Processing**: Handles multiple images in batches to optimize memory usage and speed
# @markdown - **Consistent Preprocessing**: All images resized to 220×220 (optimal for ResNet50) and normalized using ResNet's specific preprocessing
# @markdown - **Feature Extraction**: Uses the pre-trained ResNet50 base model to convert images to meaningful feature vectors

from keras.applications.resnet import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
def process_image_batch(image_paths, batch_size=32): # """Process multiple images in batches"""

    images = []
    for path in image_paths:
        img = load_img(path, target_size=(220, 220))
        img_array = img_to_array(img)
        images.append(img_array)

    images = np.array(images)
    images = preprocess_input(images)
    features = base_model.predict(images, batch_size=batch_size, verbose=0)
    return features
  # Returns extracted features for downstream tasks

In [ ]:
# @title # Extract features using parallel processing
# @markdown ⚡ **Efficient Feature Extraction Pipeline**
# @markdown
# @markdown - **Parallel-Ready Processing**: Designed for batch processing to maximize CPU/GPU utilization
# @markdown - **Smart File Handling**: Automatically filters valid image formats (.jpg, .jpeg, .png) and skips invalid paths
# @markdown - **Memory Optimization**: Processes images in batches (default=32) to balance speed and resource usage
# @markdown - **Progress Tracking**: Integrated tqdm progress bar for real-time monitoring
# @markdown - **Label Preservation**: Maintains perfect alignment between features and labels throughout processing

from keras.applications.resnet import preprocess_input
from keras.preprocessing.image import load_img, img_to_array # Import necessary functions
import numpy as np
from tqdm import tqdm
import os
def extract_features(category_paths):  # """Extract features using parallel processing"""

    all_image_paths = []
    all_labels = []

    # Collect all valid image paths and labels
    for label, path in category_paths.items():
        if os.path.exists(path):
            image_files = [os.path.join(path, f) for f in os.listdir(path)
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            all_image_paths.extend(image_files)
            all_labels.extend([label] * len(image_files))

    # Process images in batches
    batch_size = 32
    features_list = []

    for i in tqdm(range(0, len(all_image_paths), batch_size)):
        batch_paths = all_image_paths[i:i + batch_size]
        batch_features = process_image_batch(batch_paths, batch_size)
        features_list.append(batch_features.reshape(batch_features.shape[0], -1))

    return np.vstack(features_list), np.array(all_labels)

# Extract features with progress bar

features, labels = extract_features(emotion_paths)


100%|██████████| 47/47 [00:40<00:00,  1.17it/s]


In [ ]:
# @title # Save features and labels
# @markdown 💾 **saving time**
# @markdown
# @markdown - **Time-Saving Storage**: Avoid reprocessing by saving extracted features (saves hours of computation)

np.save('emotions_features.npy', features)
np.save('emotions_labels.npy', labels)

# @markdown ```python
# @markdown import numpy as np
# @markdown np.save('emotions_features.npy', features)
# @markdown np.save('emotions_labels.npy', labels)
# @markdown ```


In [ ]:
# @title # Splitting data into train/test sets
# @markdown 🧮 **Strategic Data Partitioning**
# @markdown
# @markdown - **Balanced Splitting**: 18% test size (82% training) for optimal evaluation
# @markdown - **Stratified Sampling**: Preserves class distribution in both sets (critical for imbalanced data)
# @markdown - **Reproducibility**: Fixed random_state=42 ensures consistent splits across runs
# @markdown - **Shuffling**: Prevents ordered data artifacts

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.18, random_state=42,
    stratify=labels, shuffle=True
)

# @markdown 🔍 **Splitting data into train/test sets**:
# @markdown ```python
# @markdown from sklearn.model_selection import train_test_split
# @markdown X_train, X_test, y_train, y_test = train_test_split(
# @markdown features,
# @markdown labels,
# @markdown test_size=0.18,     # 18% for testing (adjust based on dataset size)
# @markdown random_state=42,    # Ensures reproducibility
# @markdown stratify=labels ,   # Maintains original class ratios
# @markdown shuffle=True )      # Avoids ordered data bias
# @markdown ```

In [ ]:
# @title # Support Vector Machine classifier algorithm
# @markdown 🎯 **Why SVM?**
# @markdown - Old but Gold Choice   and i can handle it
# @markdown - Works well with extracted features
# @markdown - Less prone to overfitting

from sklearn.svm import SVC

# Initialize and train the model
emot_model = SVC(
    kernel='linear',   # Optimal per our experiments
    random_state=42    # For reproducibility
)

emot_model.fit(X_train, y_train)



# @markdown ```python
# @markdown from sklearn.svm import SVC
# @markdown emot_model = SVC(
# @markdown     kernel='linear',
# @markdown     random_state=42)
# @markdown emot_model.fit(X_train, y_train)
# @markdown ```





SVC(kernel='linear', random_state=42)

In [ ]:
# @title # Evaluate Model Performance
# @markdown 📊 **Key Evaluation Metrics**
# @markdown - **Accuracy**: Overall correct prediction rate
# @markdown - **Precision/Recall**: Class-specific performance
# @markdown - **F1-Score**: Balance between precision and recall

from sklearn.metrics import accuracy_score, classification_report

# Generate predictions
y_pred = emot_model.predict(X_test)

# Calculate and display metrics

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# @markdown 🔍 **Interpreting Results**:
# @markdown - Focus on both overall accuracy and class-wise metrics
# @markdown - Compare with your baseline expectations
# @markdown - Check for any class imbalance issues

# @markdown ```python
# @markdown   from sklearn.metrics import accuracy_score, classification_report
# @markdown   y_pred = emot_model.predict(X_test)
# @markdown   accuracy = accuracy_score(y_test, y_pred)
# @markdown   print(f"Model Accuracy: {accuracy * 100:.2f}%\n")
# @markdown   print("Detailed Classification Report:")
# @markdown   print(classification_report(y_test, y_pred))
# @markdown ```


Accuracy: 93.98%

Classification Report:
              precision    recall  f1-score   support

       Angry       0.91      0.86      0.88        56
       Happy       0.93      0.96      0.95        55
        fear       0.93      0.93      0.93        56
     neutral       0.95      0.98      0.97        43
    surprise       0.98      0.98      0.98        56

    accuracy                           0.94       266
   macro avg       0.94      0.94      0.94       266
weighted avg       0.94      0.94      0.94       266



In [ ]:
# @title # Save Model
# @markdown 💾 **Model Persistence**
# @markdown - Save trained model to avoid retraining
# @markdown - Preserves all learned parameters
# @markdown - Enables quick deployment/reuse

import joblib


joblib.dump(emot_model, 'emot_model.pkl')
print("✅ Model saved successfully as 'emot_model.pkl'")

# @markdown 🔄 **To Load Later**:
# @markdown ```python
# @markdown joblib.dump(emot_model, 'emot_model.pkl')
# @markdown print("✅ Model saved successfully as 'emot_model.pkl'")
# @markdown ```

✅ Model saved successfully as 'emot_model.pkl'
